<a href="https://colab.research.google.com/github/rafid29mehda/User-Review-Analyzer/blob/main/Label_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1

In [ ]:
# Install required packages
!pip install transformers
!pip install datasets
!pip install torch
!pip install pandas
!pip install scikit-learn  # Correct package name
!pip install tqdm

# Install the compatible version of pyarrow to avoid conflicts
!pip install pyarrow==14.0.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 40.1 MB/s

Step 2


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
from tqdm import tqdm


Step 3, 4

In [ ]:
from google.colab import files
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import Dataset

# Load PROMISE_exp1.csv from your Google Drive
uploaded = files.upload()

# Load the PROMISE dataset
promise_df = pd.read_csv('PROMISE_exp1.csv')

# Map labels to integers (F -> 0, NF -> 1)
promise_df['_class_'] = promise_df['_class_'].map({'F': 0, 'NF': 1})

# Split the data into training and validation sets
train_df, val_df = train_test_split(promise_df, test_size=0.2, random_state=42)

# Create Dataset objects
train_dataset = Dataset.from_pandas(train_df[['RequirementText', '_class_']])
val_dataset = Dataset.from_pandas(val_df[['RequirementText', '_class_']])

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize the dataset and rename '_class_' to 'labels'
def tokenize_function(examples):
    return tokenizer(examples['RequirementText'], padding="max_length", truncation=True)

# Apply the tokenizer to the dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Rename '_class_' to 'labels' to make it compatible with Trainer
train_dataset = train_dataset.rename_column('_class_', 'labels')
val_dataset = val_dataset.rename_column('_class_', 'labels')

# Set the format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Saving PROMISE_exp1.csv to PROMISE_exp1.csv


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/775 [00:00<?, ? examples/s]

Map:   0%|          | 0/194 [00:00<?, ? examples/s]

Step 5

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",  # Use 'eval_strategy' instead of 'evaluation_strategy'
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.564000,0.575811


Step 6

In [ ]:
# Save the model and tokenizer
model.save_pretrained('fine-tuned-bert')
tokenizer.save_pretrained('fine-tuned-bert')


('fine-tuned-bert/tokenizer_config.json',
 'fine-tuned-bert/special_tokens_map.json',
 'fine-tuned-bert/vocab.txt',
 'fine-tuned-bert/added_tokens.json')

Step 7

In [ ]:
# Load reviews.csv from your Google Drive
uploaded = files.upload()

# Load the reviews dataset
reviews_df = pd.read_csv('reviews.csv')

# Load the fine-tuned model and tokenizer
model = BertForSequenceClassification.from_pretrained('fine-tuned-bert')
tokenizer = BertTokenizer.from_pretrained('fine-tuned-bert')

# Function to classify reviews
def classify_review(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return 'F' if predicted_class == 0 else 'NF'

# Apply classification to the reviews
tqdm.pandas()
reviews_df['RequirementType'] = reviews_df['content'].progress_apply(classify_review)

# Save the labeled reviews
reviews_df.to_csv('labeled_reviews.csv', index=False)
print("Labeled reviews saved to 'labeled_reviews.csv'.")


Saving reviews.csv to reviews.csv


100%|██████████| 12495/12495 [33:03<00:00,  6.30it/s]


Labeled reviews saved to 'labeled_reviews.csv'.


Step 8

In [ ]:
from google.colab import files
files.download('labeled_reviews.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>